In [84]:
import math
import numpy as np
import altair as alt
import pandas as pd
from scipy.stats import uniform, norm
from statsmodels.distributions.empirical_distribution import ECDF
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:
fname = '../data/fijiquakes.dat'

In [3]:
df = pd.read_csv(fname, sep="\s+")

In [4]:
magnitudes = df['mag']

In [19]:
n = len(magnitudes)

In [21]:
alpha = 0.05

In [22]:
epsilon = math.sqrt((1/(2*n)) * math.log(2/alpha))

In [23]:
def U_closure(estimated_cdf, epsilon_n):

    def U(x):
        return min(estimated_cdf(x) + epsilon_n, 1)

    return U

In [24]:
def L_closure(estimated_cdf, epsilon_n):

    def L(x):
        return max(estimated_cdf(x) - epsilon_n, 0)

    return L

In [25]:
def estimated_cdf_closure(data):

    def estimated_cdf(x):
        return np.count_nonzero(data <= x) / len(data)

    return estimated_cdf

In [26]:
magnitude_estimated_cdf = estimated_cdf_closure(magnitudes)

In [28]:
u_function = U_closure(magnitude_estimated_cdf, epsilon)

In [29]:
l_function = L_closure(magnitude_estimated_cdf, epsilon)

In [31]:
sorted_magnitudes = sorted(magnitudes)

In [73]:
# X = np.linspace(3, 7, 50)
X = sorted(magnitudes)

In [74]:
upper_band_data = pd.DataFrame({
        'x': X,
        'y': [u_function(x) for x in X]
        })

In [75]:
lower_band_data = pd.DataFrame({
        'x': X,
        'y': [l_function(x) for x in X]
        })

In [76]:
estimated_cdf_data = pd.DataFrame({
        'x': X,
        'y': [magnitude_estimated_cdf(x) for x in X]
        })

In [77]:
upper_band_chart = alt.Chart(upper_band_data).mark_line(color='red').encode(
        x=alt.X('x', axis=alt.Axis( title='x')),
        y=alt.Y('y', axis=alt.Axis(title='U(x)')),
    )

In [78]:
lower_band_chart = alt.Chart(lower_band_data).mark_line(color='blue').encode(
        x=alt.X('x', axis=alt.Axis( title='x')),
        y=alt.Y('y', axis=alt.Axis(title='L(x)')),
    )

In [79]:
estimated_cdf_chart = alt.Chart(estimated_cdf_data).mark_line(color='black').encode(
        x=alt.X('x', axis=alt.Axis( title='x')),
        y=alt.Y('y', axis=alt.Axis(title='F_hat(x)')),
    )

In [80]:
upper_band_chart + lower_band_chart + estimated_cdf_chart

alt.LayerChart(...)

In [93]:
F_a = magnitude_estimated_cdf(4.3)
F_b = magnitude_estimated_cdf(4.9)

theta = F_b - F_a

In [94]:
se = math.sqrt(theta * (1-theta) / n)

In [95]:
z_95 = norm.ppf(0.975)

In [96]:
print(((theta - z_95 * se), (theta + z_95 * se)))

(0.49505217489045894, 0.5569478251095411)
